In [2]:
import sys
sys.path.append("./")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")

from typing import TYPE_CHECKING
if TYPE_CHECKING:
    from snakemake_stub import *
from Bio import SeqIO
import scipy.sparse as sp
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors  
import numpy as np
from sim_meta_function import load_reads,build_kmer_index,build_feature_matrix,get_table,get_simhash,evaluate

ref_database = '/home/miaocj/docker_dir/data/metagenome/bacteria/part1.fa'
query_reads = '/home/miaocj/docker_dir/data/metagenome/bacteria/pbsim_ONT_98_30k_10dep_part1_reads.fasta'

def init_reverse_complement():
    TRANSLATION_TABLE = str.maketrans("ACTGactg", "TGACtgac")

    def reverse_complement(sequence: str) -> str:
        """
        >>> reverse_complement("AATC")
        'GATT'
        >>> reverse_complement("CCANT")
        'ANTGG'
        """
        sequence = str(sequence)
        return sequence.translate(TRANSLATION_TABLE)[::-1]

    return reverse_complement
reverse_complement = init_reverse_complement()

k=100000
split_ref = []
ref_reads_tax_list = []
with open(ref_database, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        tax=record.description.split(' ')[1]+record.description.split(' ')[2]
        for p in range(0,len(record.seq) - k + 1,50000):
            kmer = record.seq[p : p + k]
            split_ref.append(kmer)
            ref_reads_tax_list.append(tax)
            split_ref.append(reverse_complement(kmer))
split_ref_tax_dict = {i:tax for i,tax in enumerate(ref_reads_tax_list)}

ref_reads_tax_list = []
with open(ref_database) as file:
    for lines in file:
        if lines[0] == '>':
            line = lines.strip().split(' ')
            ref_reads_tax_list.append(line[1]+line[2])
ref_read_tax = {i:tax for i,tax in enumerate(ref_reads_tax_list)}

flag = 0
que_read_tax = {}
with open(query_reads) as file:
    for lines in file:
        if lines[0] == '>':
            start = lines.index('S')
            end = lines.index('_')
            ref_num = lines[start+1:end]
            que_read_tax[flag] = ref_read_tax[int(ref_num)-1]
            flag +=1
print("loading done\nbuilding kmer index")
sample_fraction=0.1
min_multiplicity=2
seed=562104830
kmer_indices = build_kmer_index(
        read_sequences=split_ref,
        k=16,
        sample_fraction=sample_fraction,
        min_multiplicity=min_multiplicity,
        seed=seed)
print("done\nloading query reads")
qread_names, qread_orientations, qread_sequences = load_reads(query_reads)

print("done\nbuilding feature matrix")

ref_feature_matrix,ref_read_features = build_feature_matrix(read_sequences=split_ref,kmer_indices=kmer_indices,k=16)
que_feature_matrix,que_read_features = build_feature_matrix(read_sequences=qread_sequences,kmer_indices=kmer_indices,k=16)
kmer_num = que_feature_matrix.shape[1]
print("done\nbuilding hash table")

loading done
building kmer index
Counter({2: 66553881, 4: 7903925, 1: 2368196, 6: 974758, 8: 273618, 3: 213990, 10: 57248, 5: 55512, 7: 16132, 9: 5414})
done
loading query reads
done
building feature matrix
done
building hash table


In [7]:
ref_feature_matrix.shape

(1804, 14482315)

In [4]:
# repeat=60
# new = np.load('/home/miaocj/docker_dir/data/metagenome/hash_table.npz')
# hash_table_bool = new['arr_0'][:kmer_num]
# hash_table2 = hash_table_bool.astype(int)  
# hash_table = np.where(hash_table2 == 0, -1, 1)

print("done\nget ref simhash")
ref_reads_simhash_array = get_simhash(ref_read_features,hash_table)
print("ref done\nget que simhash")
que_reads_simhash_array = get_simhash(que_read_features,hash_table) ##修改simhash的计算 去掉concate
print("que done\nget neignbors")
def hamming_distance(x, y):
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric=hamming_distance)
nbrs.fit(ref_reads_simhash_array)
indices = nbrs.kneighbors(que_reads_simhash_array,return_distance=False)
print("done\nevaluates")

##evaluate
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices,split_ref_tax_dict,que_read_tax)
print(precision,sensitivity,precision_sep,sensitivity_sep)

done
get ref simhash
ref done
get que simhash
que done
get neignbors
done
evaluates
0.9898351069527734 0.9895657254201538 [0.9885103  0.99408721 0.99716178 0.97678221 0.99534247 0.99420443
 0.98779637 0.98921569 0.99586777 0.97588832 0.99332963] [0.99720224 0.9962963  0.99246704 0.97806156 0.97871767 0.98589342
 0.99683322 0.9990099  0.9790625  0.98779705 0.99388209]


In [6]:
sensitivity_sep

array([0.99720224, 0.9962963 , 0.99246704, 0.97806156, 0.97871767,
       0.98589342, 0.99683322, 0.9990099 , 0.9790625 , 0.98779705,
       0.99388209])

In [3]:
kmer_num = que_feature_matrix.shape[1]
new = np.load('/home/miaocj/docker_dir/data/metagenome/hash_table.npz')
hash_table_bool = new['arr_0'][:kmer_num]
hash_table2 = hash_table_bool.astype('int8')  
hash_table = np.where(hash_table2 == 0, -1, 1)

In [ ]:
hash_table_bool.shape

In [23]:
ref_reads_simhash_array = get_simhash(ref_read_features,hash_array)
print("ref done\nget que simhash")
que_reads_simhash_array = get_simhash(que_read_features,hash_array) ##修改simhash的计算 去掉concate

print("que done\nget neignbors")
def hamming_distance(x, y):
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric=hamming_distance)
nbrs.fit(ref_reads_simhash_array)
indices = nbrs.kneighbors(que_reads_simhash_array,return_distance=False)
print("done\nevaluates")

##evaluate
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices,split_ref_tax_dict,que_read_tax)
print(precision,sensitivity,precision_sep,sensitivity_sep)

ref done
get que simhash
que done
get neignbors
done
evaluates
0.9981726870494413 0.9985136609918157 [0.9988024  0.99925981 1.         0.99478827 1.         0.99739448
 0.99754213 0.9967925  0.99843994 0.9974359  0.99944414] [1.         1.         1.         1.         0.9846444  1.
 0.99964814 1.         1.         0.99935774 1.        ]


In [15]:
np.savez('/home/miaocj/docker_dir/data/metagenome/hash_table.npz',hash_table)

In [35]:
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', metric=hamming_distance)
nbrs.fit(ref_reads_simhash_array)
indices = nbrs.kneighbors(que_reads_simhash_array,return_distance=False)